In [1]:
import pyspark 
from pyspark.sql  import SparkSession 
from pyspark.sql.functions import collect_list, col, lit 
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, TimestampType, DateType, ArrayType
import os 
import json 
from pyspark.sql.functions import to_date, when

In [2]:
aws_s3_endpoint= "http://ibm-lh-lakehouse-minio-svc.cpd.svc.cluster.local:9000"
aws_access_key = "5e2fmxrFtyowmefsv2m42deC"
aws_secret_key ="sDzei2e6zsodf2zF9ccy03zb"
aws_s3_region = "us-east-1" 

# Khởi tạo SparkSession với các cấu hình MinIO
conf = (
    pyspark.SparkConf()
        .setAppName('raw_to_standard_etl')
        .set('spark.jars.packages', 
             'org.apache.iceberg:iceberg-spark-runtime-3.4_2.12:1.5.0,'
                'software.amazon.awssdk:bundle:2.24.8,'
                'org.apache.hadoop:hadoop-aws:3.3.4,'  
                'org.apache.hadoop:hadoop-common:3.3.4,'  
                'io.minio:minio:8.5.0,' 
                'org.apache.spark:spark-hadoop-cloud_2.13:3.2.1,' 
                'software.amazon.awssdk:url-connection-client:2.24.8,' 
                'com.amazonaws:aws-java-sdk-bundle:1.12.207')
        .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions')
        .set('spark.hadoop.fs.s3a.access.key', aws_access_key)
        .set('spark.hadoop.fs.s3a.secret.key', aws_secret_key)
        .set('spark.hadoop.fs.s3a.endpoint', aws_s3_endpoint)
        .set('spark.hadoop.fs.s3a.region', aws_s3_region)
        .set('spark.hadoop.fs.s3a.path.style.access', 'true')
        .set('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
        .set('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
        .set('spark.hadoop.fs.s3a.connection.ssl.enabled', 'false')
        .set('spark.hadoop.fs.s3a.aws.credentials.provider', 
             'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider,'
             'org.apache.hadoop.fs.s3a.EnvironmentVariableCredentialsProvider')
        .set('spark.sql.extensions', 
             'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,'
             'org.projectnessie.spark.extensions.NessieSparkSessionExtensions')
)
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [3]:
sc = spark.sparkContext
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", aws_access_key)
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", aws_secret_key)
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", aws_s3_endpoint)
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")

In [6]:
bucket_name = "hive-bucket"
file_name = "account"
file_path = f"s3a://{bucket_name}/{file_name}"

# Đọc tệp CSV từ MinIO
df = spark.read.csv(file_path, header = False)
df.show()


+-------+---------+---------+-------+--------------+---+-----+-----+----+----------+
|    _c0|      _c1|      _c2|    _c3|           _c4|_c5|  _c6|  _c7| _c8|       _c9|
+-------+---------+---------+-------+--------------+---+-----+-----+----+----------+
|1564506|113391457|113391457|Account|TRAN T *** YEN|  F|10001|ACTVE|1003| 3/12/2018|
| 958931|106573034|106573034|Account|NGUYEN *** ANH|  F|10001|ACTVE|1003| 7/10/2002|
| 958933|106573058|106573058|Account|LE THI *** OAN|  F|10001|ACTVE|1003| 7/10/2002|
| 860046|105279370|105279370|Account|NGUYEN *** YEN|  M|10001|ACTVE|1003|17-06-2019|
| 958932|106573046|106573046|Account|PHAN G ***  TU|  M|10001|ACTVE|1003| 7/10/2002|
| 958934|106573072|106573072|Account|LE KIM *** OAN|  F|10001|ACTVE|1003| 7/10/2002|
| 860047|105279435|105279435|Account|LE ANH ***  TU|  M|10001|ACTVE|1003| 10/2/2017|
|2008144|119822032|119822032|Account|HO VAN *** DAT|  M|10001|ACTVE|1003|27-09-2018|
+-------+---------+---------+-------+--------------+---+-----+---